In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET='TINYAES128C'
SS_VER='SS_VER_2_1'

In [2]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Found ChipWhisperer😍


In [3]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 6163 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6163 bytes


In [4]:
sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]

In [5]:
import ctypes
import time
import numpy as np
from tqdm.notebook import trange
from picosdk.ps2000a import ps2000a as ps
from picosdk.functions import adc2mV, assert_pico_ok

# ============
# Global flags and parameters
# ============

status = {}
# This flag will be flipped to stop streaming when the trigger is detected.
autoStopOuter = True

# Number of acquisitions (traces) to capture
NUM_TRACES = 250

# Number of samples per acquisition (adjust as needed)
NUM_SAMPLES = 5000

# Sample interval: set to 100 µs (note: using PicoScope SDK, the interval is passed as an integer value
# and then multiplied by 1000 to get ns; 100*1000 = 100000 ns = 100 µs)
sample_interval = ctypes.c_int32(100)
sample_units = ps.PS2000A_TIME_UNITS['PS2000A_US']
downsample_ratio = ctypes.c_uint32(1)  # no downsampling
ratio_mode = ps.PS2000A_RATIO_MODE['PS2000A_RATIO_MODE_NONE']

In [6]:
# ============
# Define the streaming callback function.
# This function will be called repeatedly by the PicoScope during streaming.
# When a trigger is detected (triggered==True) we flip the flag to exit the capture loop.
# ============
def streaming_callback(handle, noOfSamples, startIndex, overflow, triggerAt, triggered, autoStop, pParameter):
    global autoStopOuter, bypass, traces_list, bufferA, collected_samples
    if triggered or bypass:
        destEnd = collected_samples + noOfSamples
        sourceEnd = startIndex + noOfSamples
        # Ensure we don't exceed 10,000 samples
        #if destEnd > NUM_SAMPLES:
         #   noOfSamples = NUM_SAMPLES - collected_samples  # Trim to fit
        traces_list = np.concatenate((traces_list, bufferA[startIndex:sourceEnd]))
        autoStopOuter = False
        bypass = True
        collected_samples += noOfSamples
    return 0  # must return an integer

# Convert the Python callback into a C function pointer.
cStreamingReady = ps.StreamingReadyType(streaming_callback)

In [7]:
ktp = cw.ktp.Basic()
trace_array = []
textin_array = []

key, text = ktp.next()

target.set_key(key)
print(key)

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')


In [8]:
# Open the PicoScope unit
chandle = ctypes.c_int16()
status["openunit"] = ps.ps2000aOpenUnit(ctypes.byref(chandle), None)
assert_pico_ok(status["openunit"])
print("PicoScope opened successfully.")


#For converting ADC values to mV later on
maxADC = ctypes.c_int16()
status["maximumValue"] = ps.ps2000aMaximumValue(chandle, 
                                                ctypes.byref(maxADC))
assert_pico_ok(status["maximumValue"])
print(maxADC)

PicoScope opened successfully.
c_short(32512)


In [9]:
# Set up Channel A for the AC-coupled 50 mV probe.
# (Adjust the constant names if necessary for your SDK version.)
status["setChA"] = ps.ps2000aSetChannel(chandle,
                                        ps.PS2000A_CHANNEL['PS2000A_CHANNEL_A'],
                                        1,  # enabled
                                        ps.PS2000A_COUPLING['PS2000A_AC'],  # AC coupling
                                        ps.PS2000A_RANGE['PS2000A_50MV'],  # 50 mV range
                                        0)  # no offset
assert_pico_ok(status["setChA"])
print("Channel A configured: AC, 50 mV range.")

# Set up Channel B to be used as trigger at 2 V.
status["setChB"] = ps.ps2000aSetChannel(chandle,
                                        ps.PS2000A_CHANNEL['PS2000A_CHANNEL_B'],
                                        1,  # enabled
                                        ps.PS2000A_COUPLING['PS2000A_DC'],
                                        ps.PS2000A_RANGE['PS2000A_2V'],  # 2 V range
                                        0)
assert_pico_ok(status["setChB"])
print("Channel B configured: DC, 2 V range for triggering.")


# Set up a simple trigger on Channel B.
# (Threshold is given in mV; 2 V = 2000 mV.)
status["trigger"] = ps.ps2000aSetSimpleTrigger(chandle,
                                                1,  # enable trigger
                                                ps.PS2000A_CHANNEL['PS2000A_CHANNEL_B'],  # trigger source: Channel B
                                                ctypes.c_int16(29260),  # threshold in mV
                                                ps.PS2000A_THRESHOLD_DIRECTION['PS2000A_RISING'],  # trigger on rising edge
                                                0,  # no delay
                                                0)  
assert_pico_ok(status["trigger"])
print("Trigger configured on Channel B (threshold 1.8 V, rising edge).")

# Allocate a buffer for Channel A.
# We create a NumPy array of type int16 (as per the PicoScope sample data type).
bufferA = np.zeros(NUM_SAMPLES, dtype=np.int16)
memory_segment = 0
# Register the data buffer with the PicoScope.
status["setDataBufferA"] = ps.ps2000aSetDataBuffer(chandle,
                                                    ps.PS2000A_CHANNEL['PS2000A_CHANNEL_A'],
                                                    bufferA.ctypes.data_as(ctypes.POINTER(ctypes.c_int16)),
                                                    NUM_SAMPLES,
                                                    memory_segment,
                                                    ratio_mode)
                                                    
assert_pico_ok(status["setDataBufferA"])
print("Data buffer for Channel A allocated ({} samples).".format(NUM_SAMPLES))



# Prepare list to collect acquired traces.
traces_array = []

print("Beginning capture loop ({} acquisitions)...".format(NUM_TRACES))
for trace_num in trange(NUM_TRACES):
    # Reset the streaming flag.
    autoStopOuter = True
    bypass = False
    traces_list = np.array([])
    collected_samples = 0
    
    # Start streaming mode.
    status["runStreaming"] = ps.ps2000aRunStreaming(chandle,
                                                    ctypes.byref(sample_interval),
                                                    sample_units,
                                                    0,
                                                    NUM_SAMPLES,
                                                    True,
                                                    downsample_ratio,
                                                    ratio_mode,
                                                    NUM_SAMPLES)
    assert_pico_ok(status["runStreaming"])
    target.simpleserial_write('p', text)
    # Poll for streaming data until the trigger is detected.
    while collected_samples < NUM_SAMPLES:
        status["getStreamingLatestValues"] = ps.ps2000aGetStreamingLatestValues(chandle,
                                                                                cStreamingReady,
                                                                                None)
        time.sleep(0.01)
    # Once trigger is detected, stop streaming.
    status["stop"] = ps.ps2000aStop(chandle)
    assert_pico_ok(status["stop"])
    response = target.simpleserial_read('r', 16)
    textin_array.append(text)
    traces_array.append(traces_list)
    
    key, text = ktp.next() 

mv_array = adc2mV(traces_array, 
                  ps.PS2000A_RANGE['PS2000A_50MV'], 
                  maxADC)
traces_pico = np.array(mv_array)
print("Capture complete. Data shape: ", traces_pico.shape)




Channel A configured: AC, 50 mV range.
Channel B configured: DC, 2 V range for triggering.
Trigger configured on Channel B (threshold 1.8 V, rising edge).
Data buffer for Channel A allocated (5000 samples).
Beginning capture loop (250 acquisitions)...


  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
scope.dis()
target.dis()
# Close the PicoScope device.
status["close"] = ps.ps2000aCloseUnit(chandle)
assert_pico_ok(status["close"])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(traces_pico[4])
plt.show()

In [ ]:
from WPI_SCA_LIBRARY.CWScope import *
from WPI_SCA_LIBRARY.FileFormat import *
from WPI_SCA_LIBRARY.DPA import *
from WPI_SCA_LIBRARY.LeakageModels import *
from WPI_SCA_LIBRARY.Metrics import *

fp = FileParent("Testfile", ".", False)

In [ ]:
def calculate_diffs(guess, byteindex=0, bitnum=0):
    """Perform a simple DPA on two traces, uses global `textin_array` and `trace_array` """
    
    one_list = []
    zero_list = []

    for trace_index in range(NUM_TRACES):
        hypothetical_leakage = aes_internal(guess, textin_array[trace_index][byteindex])

        #Mask off the requested bit, most likely LSB
        if hypothetical_leakage & (1<<bitnum):
            one_list.append(traces_pico[trace_index])
        else:
            zero_list.append(traces_pico[trace_index])

    one_avg = np.asarray(one_list).mean(axis=0)
    zero_avg = np.asarray(zero_list).mean(axis=0)
    return abs(one_avg - zero_avg)

In [ ]:
#Store your key_guess here, compare to known_key
key_guess = []
known_key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6, 0xab, 0xf7, 0x15, 0x88, 0x09, 0xcf, 0x4f, 0x3c]

for subkey in trange(0, 16, desc="Attacking Subkey"): #aka each byte
    max_diffs = [0]*256
    full_diffs = [0]*256
    for guess in range(0, 256):
        full_diff_trace = calculate_diffs(guess, subkey)
        max_diffs[guess] = np.max(full_diff_trace)
        full_diffs[guess] = full_diff_trace
        
    #Get argument sort, as each index is the actual key guess.
    sorted_args = np.argsort(max_diffs)[::-1]
    
    #Keep most likely
    key_guess.append(sorted_args[0])
    
    #Print results
    print("Subkey %2d - most likely %02X (actual %02X)"%(subkey, key_guess[subkey], known_key[subkey]))
    
    #Print other top guesses
    print(" Top 5 guesses: ")
    for i in range(0, 5):
        g = sorted_args[i]
        print("   %02X - Diff = %f"%(g, max_diffs[g]))
    
    print("\n")